# Rf 6 0 1_Intminuit
Likelihood and minimization: interactive minimization with MINUIT



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:57 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooConstVar.h"
#include "RooProdPdf.h"
#include "RooAddPdf.h"
#include "RooMinimizer.h"
#include "RooFitResult.h"
#include "RooPlot.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "TH1.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Setup pdf and likelihood
 -----------------------------------------------

Observable

In [3]:
RooRealVar x("x", "x", -20, 20);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Model (intentional strong correlations)

In [4]:
RooRealVar mean("mean", "mean of g1 and g2", 0);
RooRealVar sigma_g1("sigma_g1", "width of g1", 3);
RooGaussian g1("g1", "g1", x, mean, sigma_g1);

RooRealVar sigma_g2("sigma_g2", "width of g2", 4, 3.0, 6.0);
RooGaussian g2("g2", "g2", x, mean, sigma_g2);

RooRealVar frac("frac", "frac", 0.5, 0.0, 1.0);
RooAddPdf model("model", "model", RooArgList(g1, g2), frac);

Generate 1000 events

In [5]:
RooDataSet *data = model.generate(x, 1000);

Construct unbinned likelihood of model w.r.t. data

In [6]:
RooAbsReal *nll = model.createNLL(*data);

Interactive minimization, error analysis
 -------------------------------------------------------------------------------

Create minuit interface object

In [7]:
RooMinimizer m(*nll);

Activate verbose logging of minuit parameter space stepping

In [8]:
m.setVerbose(kTRUE);

Call migrad to minimize the likelihood

In [9]:
m.migrad();

[#0] WARNING:Minization -- RooMinimizerFcn::synchronize: WARNING: no initial error estimate available for frac: using 0.1
[#0] WARNING:Minization -- RooMinimizerFcn::synchronize: WARNING: no initial error estimate available for sigma_g2: using 0.3
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 frac         5.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     2 sigma_g2     4.00000e+00  3.00000e-01    3.00000e+00  6.00000e+00
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **    6 **MIGRAD        1000           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.

prevFCN = 2660.220684   START MIGRAD MINIMIZATION.  STRATEGY 

Print values of all parameters, that reflect values (and error estimates)
 that are back propagated from MINUIT

In [10]:
model.getParameters(x)->Print("s");

  1) RooRealVar::     frac = 0.6236 +/- 0.164
  2) RooRealVar::     mean = 0
  3) RooRealVar:: sigma_g1 = 3
  4) RooRealVar:: sigma_g2 = 4.115 +/- 0.405


Disable verbose logging

In [11]:
m.setVerbose(kFALSE);

Run hesse to calculate errors from d2l/dp2

In [12]:
m.hesse();

 **********
 **    7 **SET ERR         0.5
 **********
 **********
 **    8 **SET PRINT           1
 **********
 **********
 **    9 **HESSE        1000
 **********
 COVARIANCE MATRIX CALCULATED SUCCESSFULLY
 FCN=2659.74 FROM HESSE     STATUS=OK             10 CALLS          47 TOTAL
                     EDM=4.80672e-05    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                INTERNAL      INTERNAL  
  NO.   NAME      VALUE            ERROR       STEP SIZE       VALUE   
   1  frac         6.23602e-01   1.64221e-01   1.05899e-03   2.49794e-01
   2  sigma_g2     4.11458e+00   4.05667e-01   8.74227e-04  -2.59859e-01
                               ERR DEF= 0.5
 EXTERNAL ERROR MATRIX.    NDIM=  25    NPAR=  2    ERR DEF=0.5
  2.807e-02  6.216e-02 
  6.216e-02  1.691e-01 
 PARAMETER  CORRELATION COEFFICIENTS  
       NO.  GLOBAL      1      2
        1  0.90238   1.000  0.902
        2  0.90238   0.902  1.000
frac=0.6236, sigma_g2=4.115, 

Print value (and error) of sigma_g2 parameter, that reflects
 value and error back propagated from MINUIT

In [13]:
sigma_g2.Print();

RooRealVar::sigma_g2 = 4.115 +/- 0.4057  L(3 - 6) 


Run minos on sigma_g2 parameter only

In [14]:
m.minos(sigma_g2);

 **********
 **   10 **MINOS        1000           2
 **********
 FCN=2659.74 FROM MINOS     STATUS=SUCCESSFUL     44 CALLS          91 TOTAL
                     EDM=4.80672e-05    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                  PARABOLIC         MINOS ERRORS        
  NO.   NAME      VALUE            ERROR      NEGATIVE      POSITIVE   
   1  frac         6.23602e-01   1.64221e-01                            
   2  sigma_g2     4.11458e+00   4.05667e-01  -3.79331e-01   4.57363e-01
                               ERR DEF= 0.5
frac=0.6236, sigma_g2=4.115, 

Print value (and error) of sigma_g2 parameter, that reflects
 value and error back propagated from MINUIT

In [15]:
sigma_g2.Print();

RooRealVar::sigma_g2 = 4.115 +/- (-0.3793,0.4574)  L(3 - 6) 


Saving results, contour plots
 ---------------------------------------------------------

Save a snapshot of the fit result. this object contains the initial
 fit parameters, the final fit parameters, the complete correlation
 matrix, the EDM, the minimized FCN , the last MINUIT status code and
 the number of times the RooFit function object has indicated evaluation
 problems (e.g. zero probabilities during likelihood evaluation)

In [16]:
RooFitResult *r = m.save();

Make contour plot of mx vs sx at 1,2,3 sigma

In [17]:
RooPlot *frame = m.contour(frac, sigma_g2, 1, 2, 3);
frame->SetTitle("Minuit contour plot");

 **********
 **   11 **SET ERR         0.5
 **********
 **********
 **   12 **SET PRINT           1
 **********
 START MNCONTOUR CALCULATION OF   50 POINTS ON CONTOUR.
 Y-AXIS: PARAMETER   2  sigma_g2
                         X-AXIS: PARAMETER   1  frac
   50 POINTS ON CONTOUR.   FMIN=  2.65974e+03   ERRDEF=        0.5
         fracsigma_g2fracsigma_g2
     1  3.97200e-01  3.75562e+00             26  7.56837e-01  4.54075e+00
     2  4.17255e-01  3.73525e+00             27  7.44618e-01  4.57195e+00
     3  4.44729e-01  3.74324e+00             28  7.25403e-01  4.55125e+00
     4  4.69531e-01  3.75750e+00             29  7.12749e-01  4.52601e+00
     5  4.92331e-01  3.77448e+00             30  7.00694e-01  4.50076e+00
     6  5.13625e-01  3.79332e+00             31  6.91094e-01  4.47447e+00
     7  5.33410e-01  3.81339e+00             32  6.70260e-01  4.42290e+00
     8  5.51977e-01  3.83459e+00             33  6.51349e-01  4.37060e+00
     9  5.69394e-01  3.85674e+00             34  6.32

Print the fit result snapshot

In [18]:
r->Print("v");


  RooFitResult: minimized FCN value: 2660, estimated distance to minimum: 4.807e-05
                covariance matrix quality: Full, accurate covariance matrix
                Status : MIGRAD=0 HESSE=0 MINOS=0 

    Constant Parameter    Value     
  --------------------  ------------
                  mean    0.0000e+00
              sigma_g1    3.0000e+00

    Floating Parameter  InitialValue    FinalValue (+HiError,-LoError)    GblCorr.
  --------------------  ------------  ----------------------------------  --------
                  frac    5.0000e-01    6.2360e-01         +/-  1.64e-01  <none>
              sigma_g2    4.0000e+00    4.1146e+00 (+4.57e-01,-3.79e-01)  <none>



Change parameter values, floating
 -----------------------------------------------------------------

At any moment you can manually change the value of a (constant)
 parameter

In [19]:
mean = 0.3;

Rerun migrad,hesse

In [20]:
m.migrad();
m.hesse();
frac.Print();

 **********
 **   17 **SET PRINT           1
 **********
 **********
 **   18 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 frac         6.23602e-01  1.64221e-01    0.00000e+00  1.00000e+00
     2 sigma_g2     4.11458e+00  4.05667e-01    3.00000e+00  6.00000e+00
 **********
 **   19 **SET ERR         0.5
 **********
 **********
 **   20 **SET PRINT           1
 **********
 **********
 **   21 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **   22 **MIGRAD        1000           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.

prevFCN = 2663.4116   START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-03
frac=0.6256, 
prevFCN = 2663.416604  frac=0.6216, 
prevFCN = 2663.407358  frac=0.6236, sigma_g2=4.119, 
prevFCN = 2663.411098  sigma_g2=4.11, 
prevFCN = 2663.412871   FCN=2663.41 FROM MIGRAD    STATUS=INITIATE 

Now fix sigma_g2

In [21]:
sigma_g2.setConstant(kTRUE);

Rerun migrad,hesse

In [22]:
m.migrad();
m.hesse();
frac.Print();

new TCanvas("rf601_intminuit", "rf601_intminuit", 600, 600);
gPad->SetLeftMargin(0.15);
frame->GetYaxis()->SetTitleOffset(1.4);
frame->Draw();

 **********
 **   26 **SET PRINT           1
 **********
 **********
 **   27 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 frac         5.62746e-01  1.97958e-01    0.00000e+00  1.00000e+00
     2 sigma_g2     3.99992e+00  3.99992e-01     no limits
 **********
 **   28 **FIX           2
 **********
 **********
 **   29 **SET ERR         0.5
 **********
 **********
 **   30 **SET PRINT           1
 **********
 **********
 **   31 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **   32 **MIGRAD        1000           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.

prevFCN = 2663.357649   START MIGRAD MINIMIZATION.  STRATEGY  1.  CONVERGENCE WHEN EDM .LT. 1.00e-03
frac=0.57, 
prevFCN = 2663.361693  frac=0.5555, 
prevFCN = 2663.361647  frac=0.5656, 
prevFCN = 2663.35829  frac=0.5599, 
prevFCN = 2663.358278   FCN=2663.36 FROM MIGRAD  

Draw all canvases 

In [23]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()